# 6. Hidden Markov Models with Theano and TensorFlow
* What is different about this process compared to old process?
    * Specifically, expectation maximization vs stochastic gradient descent. 
* Set the stage for theano example, specifically the ease of taking the gradient compared to all the moving parts of EM
* Describe each part of theano code block